# 万方文献自动化下载脚本
## Reference
https://www.jianshu.com/p/83ab33037b84
## 使用说明
- 传入参数
  - search_expression(检索式),将要查询的检索式赋值给search_expression
  - download_path(默认下载地址),将想下载到的地址赋值给download_path,会修改浏览器默认下载位置
  - page_count(开始页数)，默认会从第page_count+1页位置处开始下载
- 目前的问题
  - 下载论文数量过多后会被封禁IP

In [7]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 配置检索式
search_expression = "(中图分类号:(R246.83) or 中图分类号:(R322.41) or 中图分类号:(R473.78) or 中图分类号:(R529.8) or 中图分类号:(R739.8) or 中图分类号:(R78) or 中图分类号:(R816.98) or 中图分类号:(R828.8) or 中图分类号:(R856.78) or 中图分类号:(R988.2) or 中图分类号:(TH787) or 中图分类号:(S853.58) or 中图分类号:(TQ658.4) not 刊名:(International Journal of Oral science) not 刊名:(中华口腔医学杂志) not 刊名:(华西口腔医学杂志) not 刊名:(临床口腔医学杂志) not 刊名:(国际口腔医学杂志) not 刊名:(中华老年口腔医学杂志) not 刊名:(中国实用口腔科杂志) not 刊名:(实用口腔医学杂志) not 刊名:(口腔疾病防治) not 刊名:(上海口腔医学) not 刊名:(口腔医学研究) not 刊名:(中华口腔正畸学杂志) not 刊名:(口腔医学) not 刊名:(口腔颌面修复学杂志) not 刊名:(中国口腔种植学杂志) not 刊名:(口腔材料器械杂志) not 刊名:(中国口腔颌面外科杂志) not 刊名:(北京口腔医学) not 刊名:(口腔生物医学) not 刊名:(口腔颌面外科杂志) not 刊名:(现代口腔医学杂志) not 刊名:(中华口腔医学研究杂志(电子版)) not 刊名:(全科口腔医学电子杂志) not 刊名:(牙体牙髓牙周病学杂志)) and 发表时间:2003-2024"

# 设置默认论文下载路径
download_path = "D:\\Desktop\\论文自动化下载\\万方论文"
#开始页码
page_count=51
#proxy = "your_proxy_ip:port"
# 设置浏览器选项
options = Options()
options.add_argument("--start-maximized")
#options.add_argument(f'--proxy-server={proxy}')
options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# 启动浏览器
browser = webdriver.Edge(options=options)

# 打开网页
browser.get("https://s.wanfangdata.com.cn/advanced-search/paper")

# 执行检索
professional_search_button = browser.find_element(By.XPATH, "//span[text()='专业检索']")
professional_search_button.click()
text_input = browser.find_element(By.XPATH, "//*[@id='queryInput']/textarea")
text_input.send_keys(search_expression)
search_button = browser.find_element(By.XPATH, "/html/body/div[5]/div/div[2]/div[2]/div[1]/div[4]/span[1]")
search_button.click()
paper_count=0
for i in range(page_count):
    # 尝试点击“下一页”按钮
    next_button = WebDriverWait(browser, 3).until(
        EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
    )
    browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
    next_button.click()
    time.sleep(1)  # 等待页面加载
# 下载并翻页
while True:
    print("Downloading page", paper_count, "...")
    try:
        # 下载当前页的所有文档
        try:
            download_spans = WebDriverWait(browser, 3).until(
                EC.visibility_of_all_elements_located((By.XPATH, "//span[@data-v-9f00e464='' and text()='下载']"))
            )
            #print("Downloading", len(download_spans), "papers...")
        except Exception:
            download_spans = []
            #print("No papers found.")
        try:
            free_download_spans = WebDriverWait(browser, 3).until(
                EC.visibility_of_all_elements_located((By.XPATH, "//span[@data-v-e231288e='' and text()='下载']"))
            )
            #print("Downloading", len(free_download_spans), "free papers...")
        except Exception:
            free_download_spans = []
            #print("No free papers found.")
        paper_count+=len(download_spans)+len(free_download_spans)
        for span in download_spans + free_download_spans:
            try:
                browser.execute_script("arguments[0].scrollIntoView(true);", span)
                span.click()
                time.sleep(1)  # 等待下载开始

                # 切换到新窗口并关闭
                browser.switch_to.window(browser.window_handles[-1])
                browser.close()

                # 切换回原始窗口
                browser.switch_to.window(browser.window_handles[0])
            except Exception as e:
                print("Error clicking span or closing window:", e)
        
        try:
            # 尝试点击“下一页”按钮
            next_button = WebDriverWait(browser, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
            )
            browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
            next_button.click()
        except Exception:
            break
        time.sleep(1)  # 等待页面加载
    except Exception as e:
        print("No more pages or error:", e)
        break

# 关闭浏览器
browser.quit()


KeyboardInterrupt: 

# 万方论文摘要导出
## 使用说明
- 传入参数
  - search_expression(检索式),将要查询的检索式赋值给search_expression
  - download_path(默认下载地址),将想下载到的地址赋值给download_path,会修改浏览器默认下载位置
  - page_count(开始页数)，默认会从第page_count+1页位置处开始下载
- 目前的问题
  - 下载到5500条摘要后万方有查看限制，就不会展示论文列表，无法下载

In [6]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
# 配置检索式
search_expression = "(中图分类号:(R246.83) or 中图分类号:(R322.41) or 中图分类号:(R473.78) or 中图分类号:(R529.8) or 中图分类号:(R739.8) or 中图分类号:(R78) or 中图分类号:(R816.98) or 中图分类号:(R828.8) or 中图分类号:(R856.78) or 中图分类号:(R988.2) or 中图分类号:(TH787) or 中图分类号:(S853.58) or 中图分类号:(TQ658.4) not 刊名:(International Journal of Oral science) not 刊名:(中华口腔医学杂志) not 刊名:(华西口腔医学杂志) not 刊名:(临床口腔医学杂志) not 刊名:(国际口腔医学杂志) not 刊名:(中华老年口腔医学杂志) not 刊名:(中国实用口腔科杂志) not 刊名:(实用口腔医学杂志) not 刊名:(口腔疾病防治) not 刊名:(上海口腔医学) not 刊名:(口腔医学研究) not 刊名:(中华口腔正畸学杂志) not 刊名:(口腔医学) not 刊名:(口腔颌面修复学杂志) not 刊名:(中国口腔种植学杂志) not 刊名:(口腔材料器械杂志) not 刊名:(中国口腔颌面外科杂志) not 刊名:(北京口腔医学) not 刊名:(口腔生物医学) not 刊名:(口腔颌面外科杂志) not 刊名:(现代口腔医学杂志) not 刊名:(中华口腔医学研究杂志(电子版)) not 刊名:(全科口腔医学电子杂志) not 刊名:(牙体牙髓牙周病学杂志)) and 发表时间:2003-2024"

# 设置默认论文下载路径
download_path = "D:\\Desktop"
#开始页数
page_count=0#120
#proxy = "your_proxy_ip:port"
# 设置浏览器选项
options = Options()
options.add_argument("--start-maximized")
#options.add_argument(f'--proxy-server={proxy}')
options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# 启动浏览器
browser = webdriver.Edge(options=options)

# 打开网页
browser.get("https://s.wanfangdata.com.cn/advanced-search/paper")

# 执行检索
professional_search_button = browser.find_element(By.XPATH, "//span[text()='专业检索']")
professional_search_button.click()
text_input = browser.find_element(By.XPATH, "//*[@id='queryInput']/textarea")
text_input.send_keys(search_expression)
search_button = browser.find_element(By.XPATH, "/html/body/div[5]/div/div[2]/div[2]/div[1]/div[4]/span[1]")
search_button.click()
# 等待下拉框可点击
dropdown = WebDriverWait(browser, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[1]/div[2]/div/div[1]/div[1]'))
)

browser.execute_script("arguments[0].click();", dropdown)

# 等待并选择特定的选项
option = WebDriverWait(browser, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[1]/div[2]/div/div[1]/div[2]/div[3]'))
)
browser.execute_script("arguments[0].click();", option)
paper_count=0

for i in range(page_count):
    # 尝试点击“下一页”按钮
    next_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
    )
    browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
    next_button.click()
    time.sleep(1)  # 等待页面加载
time.sleep(5)  # 等待页面加载
# 下载并翻页
while True:
    print("Downloading page", paper_count, "...")
    for i in range (1,11):
        elements=browser.find_elements(By.XPATH, "/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/div/div/label/span/input")
        for element in elements:
            browser.execute_script("arguments[0].scrollIntoView(true);", element)
            try:
                ActionChains(browser).move_to_element(element).click().perform()
            except Exception as e:
                print("Error clicking element:", e)
        try:
            # 尝试点击“下一页”按钮
            next_button = WebDriverWait(browser, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
            )
            browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
            next_button.click()
            time.sleep(3)  # 等待页面加载
        except Exception:
            break
    # 点击批量引用按钮
    export_button = browser.find_element(By.XPATH, "//span[text()='批量引用']")
    browser.execute_script("arguments[0].click();", export_button)
    original_handles = browser.window_handles
    WebDriverWait(browser, 30).until(lambda d: len(d.window_handles) > 1)
    # 切换到新窗口
    browser.switch_to.window(original_handles[-1])
    time.sleep(15)  # 等待页面加载
    tab_button = WebDriverWait(browser, 30).until(
        EC.element_to_be_clickable((By.XPATH, "//*[@id='tab-m3']"))
    )
    browser.execute_script("arguments[0].click();", tab_button)
    expport_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[7]/div[2]/div/div/div[2]/div/div[1]/div[2]"))
    )
    try:
           ActionChains(browser).move_to_element(expport_button).click().perform()
    except Exception as e:
        print("Error clicking element:", e)
    time.sleep(1)  # 等待页面加载
    # 关闭当前窗口
    browser.close()

    # 切换回原始窗口
    browser.switch_to.window(original_handles[0])
    try:
        # 尝试点击“下一页”按钮
        next_button = WebDriverWait(browser, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
        )
        browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
        next_button.click()
    except Exception:
        break
    time.sleep(3)  # 等待页面加载
    clear_button = browser.find_element(By.XPATH, "/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[1]/div[1]/div/span[2]")
    browser.execute_script("arguments[0].click();", clear_button)
    paper_count+=500
# 关闭浏览器
#browser.quit()


before open 1
after open 2
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[1]/div[1]/div/span[2]"}
  (Session info: MicrosoftEdge=131.0.2903.146); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	(No symbol) [0x00007FF75D916B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75DC3F444+1437348]
	sqlite3_dbdata_init [0x00007FF75DCE2D76+643174]
	(No symbol) [0x00007FF75D83C9DB]
	(No symbol) [0x00007FF75D83CAE3]
	(No symbol) [0x00007FF75D8792F7]
	(No symbol) [0x00007FF75D85C1DF]
	(No symbol) [0x00007FF75D833437]
	(No symbol) [0x00007FF75D876BFF]
	(No symbol) [0x00007FF75D85BE03]
	(No symbol) [0x00007FF75D832984]
	(No symbol) [0x00007FF75D831E30]
	(No symbol) [0x00007FF75D832571]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75DBEBAD4+1094964]
	(No 

# 下载摘要格式化脚本
## 使用说明
- 将"./万方论文摘要"目录下的所有txt文件解析并保存成格式化的wanfang_abstract.json

In [33]:
process_path = "./万方论文摘要"
#依次遍历目录下的文件
import os
import json
json_list = []
for root, dirs, files in os.walk(process_path):
    for file in files:
        with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
            lines = f.readlines()
        # 解析文件内容
        abstracts=[]
        for i in range(0, len(lines), 4):
            if i + 2 < len(lines):
                entry =[lines[i].strip(), lines[i + 1].strip(), lines[i + 2].strip()]
                abstracts.append(entry)

        # 输出结果
        for abstract in abstracts:
            if len(abstract) < 3:
                continue
            reference = {
                "authors": abstract[0].split(". ")[0].split(" ")[1].strip(),
                "title": abstract[0].split(". ")[1].strip() if len(abstract[0].split(". "))>1 else "",
                "journal_info": abstract[0].split(". ")[2].strip() if len(abstract[0].split(". "))>2 else "",
                "degree_grantor": abstract[1].split(": ")[1].strip() if len(abstract[1].split(": "))>1 else "",
                "abstract": abstract[2].strip().replace("摘要: ", "")
            }
            json_list.append(reference)
            f.close()
# 将解析后的数据转换为 JSON 格式
json_data = json.dumps(json_list, ensure_ascii=False, indent=4)

# 保存为文件
with open("./wanfang_abstract.json", "w", encoding="utf-8") as f:
    f.write(json_data)

print("JSON 数据已保存到 wanfang_abstract.json")

JSON 数据已保存到 wanfang_abstract.json
